In [11]:
import gym
import random
import numpy as np
import time
from gym.envs.registration import register
from IPython.display import clear_output

In [6]:
# https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py
# https://github.com/openai/gym/blob/master/gym/envs/__init__.py
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

# env_name = "CartPole-v1"
# env_name = "MountainCar-v0"
# env_name = "MountainCarContinuous-v0"
# env_name = "Acrobot-v1"
# env_name = "Pendulum-v0"
env_name = "FrozenLake-v0"
env_name = "FrozenLakeNoSlip-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
type(env.action_space)

Observation space: Discrete(16)
Action space: Discrete(4)


gym.spaces.discrete.Discrete

In [7]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [21]:
class QAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self):
        self.q_table = 1e-4*np.random.random([self.state_size, self.action_size])
        
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state,action]
        self.q_table[state,action] += self.learning_rate * q_update
        
        if done:
            self.eps = self.eps * 0.99
        
agent = QAgent(env)

Action size: 4
State size: 16


In [26]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()
        print(agent.q_table)
        time.sleep(0.05)
        clear_output(wait=True)

s: 15 a: 2
Episode: 99, Total reward: 99.0, eps: 0.006570483042414605
  (Right)
SFFF
FHFH
FFFH
HFFG
[[2.26657246e-04 5.78706808e-05 1.31003002e-01 4.79979472e-04]
 [1.04844339e-04 3.54687230e-05 2.55427402e-01 7.48193170e-04]
 [1.65884052e-04 4.36189268e-01 7.00960632e-05 8.88959016e-04]
 [1.17590145e-03 5.49928561e-05 6.38101134e-05 3.16729025e-05]
 [6.15095796e-05 4.13004395e-05 3.01916296e-05 5.90834920e-05]
 [4.25841839e-05 7.56731911e-06 8.26995681e-05 6.60735347e-05]
 [6.37688773e-05 6.48470611e-01 7.98525028e-05 3.01763264e-03]
 [5.30045086e-05 5.07371856e-05 1.99983098e-05 9.32633643e-05]
 [4.18707042e-05 5.71000704e-05 2.32037264e-04 3.18999734e-05]
 [4.15603587e-05 6.91116263e-03 2.75917720e-04 2.88792860e-05]
 [3.17844770e-04 8.42711089e-01 2.42033377e-05 2.93321581e-03]
 [9.02368275e-05 6.37119718e-05 6.00035959e-05 7.00083118e-05]
 [7.71263802e-05 1.73696049e-05 4.11883145e-05 3.22260325e-05]
 [5.22960464e-05 8.34910792e-05 1.07448006e-01 1.17173895e-04]
 [3.04657414e-03 6